<a href="https://colab.research.google.com/github/omerfcinoglu/Turkish_Sentiment_Analysis/blob/main/Turkish_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
file_path = "/content/drive/MyDrive/TSA-Dataset/train.csv"

df = pd.read_csv(file_path)
display(df.head())

,text,label,dataset
0,ürünü hepsiburadadan alalı 3 hafta oldu. orjin...,Positive,urun_yorumlari
1,"ürünlerden çok memnunum, kesinlikle herkese ta...",Positive,urun_yorumlari
2,"hızlı kargo, temiz alışveriş.teşekkür ederim.",Positive,urun_yorumlari
3,Çünkü aranan tapınak bu bölgededir .,Notr,wiki
4,bu telefonu başlıca alma nedenlerim ise elimde...,Positive,urun_yorumlari


In [9]:
test_file_path = "/content/drive/MyDrive/TSA-Dataset/test.csv"

test_df = pd.read_csv(test_file_path)
display(test_df.head())

Test data loaded successfully:


,text,label,dataset
0,Kral akbaba dikkat çekici renklere sahiptir .,Notr,wiki
1,ısrarla korkutmayı başarıyor. sanki korku çok...,Positive,HUMIR
2,Neşe ve Üzüntü köprünün kırılmaya başlamasıyla...,Notr,wiki
3,i phone 5 ten sonra gene 4'' ekranı tercih ett...,Positive,urun_yorumlari
4,Beşinci sezonda diziye yeni oyuncular katıldı .,Notr,wiki


In [11]:
num_rows_test = test_df.shape[0]
print(f"The training dataset contains {num_rows} entries.")
print(f"The test dataset contains {num_rows_test} entries.")

The training dataset contains 440679 entries.
The test dataset contains 48965 entries.


In [16]:
label_counts_df = pd.DataFrame({
    'label': label_counts.index,
    'count': label_counts.values,
    'dataset': 'Train'
})

label_counts_test_df = pd.DataFrame({
    'label': label_counts_test.index,
    'count': label_counts_test.values,
    'dataset': 'Test'
})

combined_label_counts = pd.concat([label_counts_df, label_counts_test_df], ignore_index=True)

print("Combined Label Counts DataFrame:")
display(combined_label_counts)

Combined Label Counts DataFrame:


,label,count,dataset
0,Positive,235949,Train
1,Notr,153825,Train
2,Negative,50905,Train
3,Positive,26217,Test
4,Notr,17092,Test
5,Negative,5656,Test
